<a href="https://colab.research.google.com/github/eksalailia/BigData_Genap_2022/blob/main/BigData_UAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null